In [ ]:
from collections import defaultdict

import pandas as pd
import numpy as np
import scipy.stats as ss

import lightfm
import lightfm.data as ld
import lightfm.evaluation as lv

import tqdm
import json
import glob
import faiss
#import optuna

from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as pl
import seaborn as sns

np.random.seed(31337)

In [ ]:
# Данные к семинару: https://drive.google.com/drive/folders/128GhNeYd3DTDuORdB2Hqtcd9YtKu0qd-?usp=sharing

In [ ]:
data = pd.read_csv("./data/500kfeedbacks.csv", index_col=None).drop("Unnamed: 0", axis=1)
data.head(5)

In [ ]:
positives = data[data["time"] > 0.8].copy()
positives["test"] = np.random.random(len(positives)) >= 0.7
positives.drop_duplicates(["user", "track"], inplace=True)

In [ ]:
user_counts = positives[~positives["test"]].groupby("user").size()
users = set(user_counts[user_counts >= 3].index.values)

In [ ]:
track_counts = positives[~positives["test"]].groupby("track").size()
tracks = set(track_counts[track_counts >= 3].index.values)

In [ ]:
len(users), len(tracks)

# Обучим LightFM (тут ничего нового)

In [ ]:
train_data = positives[~positives["test"] & positives["user"].isin(users) & positives["track"].isin(tracks)]
test_data = positives[positives["test"] & positives["user"].isin(users) & positives["track"].isin(tracks)]

len(train_data), len(test_data)

In [ ]:
dataset = ld.Dataset()
dataset.fit(users, tracks)

In [ ]:
train_interactions, _ = dataset.build_interactions(train_data[["user", "track"]].itertuples(index=False, name=None))
test_interactions, _ = dataset.build_interactions(test_data[["user", "track"]].itertuples(index=False, name=None))

In [ ]:
def fit_model(
    epochs=1, 
    at=10,
    loss="warp",
    no_components=30,
    learning_rate=0.01, 
    max_sampled=10,
    user_alpha=0.0, 
    item_alpha=0.0, 
    threads=30, 
    verbose=False,
    patience=3,
    epsilon=1e-6,
):
    model = lightfm.LightFM(
        no_components=no_components,
        loss=loss,
        learning_rate=learning_rate,
        max_sampled=max_sampled,
        user_alpha=user_alpha,
        item_alpha=item_alpha,
    )

    precisions_at = []
    
    for epoch in range(epochs):
        model = model.fit_partial(train_interactions, num_threads=threads)
        
        precision_at = lv.precision_at_k(model, test_interactions, train_interactions=train_interactions, k=at, num_threads=threads)
        
        if verbose:
            print(f"{epoch}:\t{np.mean(precision_at)} +/- {ss.sem(precision_at) * 1.96}")
            
        precisions_at.append(np.mean(precision_at))
            
        if epoch > patience and all([precisions_at[-j] - precisions_at[-patience-1] < epsilon for j in range(1, patience + 1)]):
            if verbose:
                print("Early stopiing!")
            break
        
    else:
        if verbose:
            print("No early stopiing happened: increase epochs maybe?")
        
    return model, precisions_at


def objective(trial):
    loss = trial.suggest_categorical("loss", ["warp", "bpr"])
    no_components = trial.suggest_categorical("no_components", [10, 30, 50])
    learning_rate = trial.suggest_categorical("learning_rate", [0.0001, 0.001, 0.01])
    max_sampled = trial.suggest_categorical("max_sampled", [10, 20, 50, 100])
    user_alpha = trial.suggest_categorical("user_alpha", [0.0, 0.0001])
    item_alpha = trial.suggest_categorical("item_alpha", [0.0, 0.0001])
    
    model, precisions_at = fit_model(
        epochs=5, 
        at=10,
        loss=loss,
        no_components=no_components, 
        learning_rate=learning_rate, 
        max_sampled=max_sampled, 
        user_alpha=user_alpha, 
        item_alpha=item_alpha,
    )
    
    return precisions_at[-1]

In [ ]:
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=30)
# best_params = study.best_params

best_params = {
    'loss': 'warp',
    'no_components': 50,
    'learning_rate': 0.01,
    'max_sampled': 100,
    'user_alpha': 0.0,
    'item_alpha': 0.0001
}

In [ ]:
model, precisions_at = fit_model(
    epochs=300,
    at=10,
    loss=best_params["loss"],
    no_components=best_params["no_components"], 
    learning_rate=best_params["learning_rate"], 
    max_sampled=best_params["max_sampled"],
    user_alpha=best_params["user_alpha"],
    item_alpha=best_params["item_alpha"],
    verbose=True,
)

In [ ]:
figure, ax = pl.subplots()

ax.plot(np.arange(len(precisions_at)), precisions_at)

pass

### Делаем маппинги индекс трека <-> айди трека и сохраняем

In [ ]:
TRACK_META = pd.read_json("./data/tracks.json", lines=True)
TRACK_META["track_index"] = TRACK_META["track"].map(lambda t: dataset.mapping()[2].get(t))
TRACK_META = TRACK_META[~np.isnan(TRACK_META["track_index"])]
TRACK_META["track_index"] = TRACK_META["track_index"].astype(int)

In [ ]:
TRACK_META

### Сохраняем эмбеддинги

In [ ]:
item_biases, item_embeddings = model.get_item_representations()
user_biases, user_embeddings = model.get_user_representations()

np.save("item_biases", item_biases)
np.save("item_embeddings", item_embeddings)
np.save("user_biases", user_biases)
np.save("user_embeddings", user_embeddings)

In [ ]:
ITEM_BIASES = np.load("item_biases.npy")
ITEM_EMBEDDINGS = np.load("item_embeddings.npy")
USER_BIASES = np.load("user_biases.npy")
USER_EMBEDDINGS = np.load("user_embeddings.npy")

### Рассчитываем "прямые" рекомендации

In [ ]:
ITEM_EMBEDDINGS_WITH_BIASES = np.concat([ITEM_BIASES[:, np.newaxis], np.ones(len(ITEM_BIASES))[:, np.newaxis], ITEM_EMBEDDINGS], axis=1)
USER_EMBEDDINGS_WITH_BIASES = np.concat([np.ones(len(USER_BIASES))[:, np.newaxis], USER_BIASES[:, np.newaxis], USER_EMBEDDINGS], axis=1)

In [ ]:
USER_RECOMMENDATIONS = USER_EMBEDDINGS_WITH_BIASES.dot(ITEM_EMBEDDINGS_WITH_BIASES.T)

In [ ]:
TOP_USER_RECOMMENDATIONS = USER_RECOMMENDATIONS.argsort(axis=-1)[:, :200]

In [ ]:
np.save("top_recommendations_raw", TOP_USER_RECOMMENDATIONS)

In [ ]:
TOP_USER_RECOMMENDATIONS = np.load("top_recommendations_raw.npy")

### Делаем маппинги индекс трека -> айди трека

In [ ]:
TRACK_META = pd.read_json("./data/tracks.json", lines=True)
TRACK_META["track_index"] = TRACK_META["track"].map(lambda t: dataset.mapping()[2].get(t))
TRACK_META = TRACK_META[~np.isnan(TRACK_META["track_index"])]
TRACK_META["track_index"] = TRACK_META["track_index"].astype(int)

In [ ]:
TRACK_META[["artist", "album", "title", "track", "track_index"]].to_csv("track_meta.csv", index=False)

In [ ]:
TRACK_META = pd.read_csv("track_meta.csv")
TRACK_META = TRACK_META.set_index("track_index")

### Делаем маппинги айди юзера -> индекс юзера

In [ ]:
import pickle

user_mapping_raw = dataset.mapping()[0]
user_mapping = {v:int(k) for k, v in user_mapping_raw.items()}

with open('user_mapping.pickle', 'wb') as f:
    pickle.dump(user_mapping, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
USER_MAPPING = dict()

with open('user_mapping.pickle', 'rb') as f:
    USER_MAPPING = pickle.load(f)

# Подход #1. Разнообразие с использованием DPP

### Алгоритм рассчета матрицы $L$ с параметризацией через $\alpha,\sigma$

https://dl.acm.org/doi/pdf/10.1145/3269206.3272018

Для пользователя $u$ и списка рекомендаций $W$,

Рассчитаем матрицу $L$,
1. Берем вектор пользователя $q_u$.
2. Забираем эмбеддинги айтемов для списка рекоммендаций $v_i, i \in W$.
3. Рассчитываем диагональные элементы матрицы $L_{ii} = (q_u, v_i), i \in W$.
4. Рассчитываем off-diagonal элементы матрицы $L_{ij} = \alpha (q_u, v_i)(q_u, v_j) exp(-\frac{(v_i, v_j)}{2\sigma^2}); i \neq j; i,j \in W$.
5. Получаем матрицу $L$.
6. Если матрица $L$ не является неотрицательно-определенной, то выполняем хак.

Хак,
1. Делаем разложение матрицы L на собственные числа (получаем собственные числа и собственные вектора ($\lambda, e$).
2. Зануляем отрицательные $\lambda$.
3. Восстанавливаем матрицу.

In [ ]:
ALPHA = 1.55
SIGMA = 0.9

# Расстояние между айдемами, в формуле D
ITEM_DISTANCES = 1 - cosine_similarity(ITEM_EMBEDDINGS) + 1e-6
# Выражение exp(-(vi, vj) / 2 sigma^2)
RBF = np.exp(-ITEM_DISTANCES / (2 * (SIGMA ** 2)))

In [ ]:
def ensure_spd(m: np.array):
    evalues, evecs = np.linalg.eig(m)
    evalues[evalues < 0] = 1e-8
    lamb = np.diag(evalues)
    return evecs @ lamb @ np.linalg.inv(evecs)
    
def compute_kernel(user_index: int, item_indices: list[int]) -> np.array:
    # В формуле (q, v)
    scores = (ITEM_EMBEDDINGS_WITH_BIASES[item_indices, :].dot(USER_EMBEDDINGS_WITH_BIASES[user_index, :]))[np.newaxis, :]
    # В формуле (q, vi)(q, vj)
    scores_outer = scores.T.dot(scores)

    # Рассчитываем матрицу L
    diag_ix = np.diag_indices(len(item_indices))
    rbf_minor = RBF[item_indices, :][:, item_indices]
    kernel = ALPHA * scores_outer * rbf_minor
    kernel[diag_ix] = scores_outer[diag_ix]
    
    return ensure_spd(kernel)

### Выполняем переранжирование с помощью DPP

Входные параметры,
* $u$ - индекс пользователя
* $W$ - список рекомендаций (индексы треков)
* $w$ - окно

0. Инициализируем R (переранжированный список рекомендаций) пустым списком.
1. Рассчитываем матрицу L с помощью алгоритма параграфом выше.
2. С помощью жадого алгоритма, строим список из $w$ переранжированных айтемов $M$ для матрицы $L[W,W]$
3. Добавляем эти айтемы в список $R$ и убираем их из списка $W$
4. Повторяем пункты 2-3 до тех пор, пока $|W| > 0$

Жадный алгоритм,

1. Создаем список ранжированных айтемов $R_w$ для окна $w$
2. Ищем такой элемент $i$, который при котором $det(L[R_w \cup i, R_w \cup i])$ максимальна.
3. Добавляем этот элемент $i$ в список $R_w$.
4. Итерируемся $w$ раз
5. Возвращаем получившийся список индексов $R_w$

https://dl.acm.org/doi/pdf/10.1145/3269206.3272018

In [ ]:
def greedy_approx_max(kernel: np.array, size: int) -> list[int]:
    assert kernel.shape[0] == kernel.shape[1], "invalid kernel size"
    items = [np.argmax(np.diag(kernel))]

    for i in range(size - 1):
        max_c = None
        max_v = None
        for c in range(kernel.shape[0]):
            if c in items:
                continue
            items_c = items + [c]
            cur_v = np.linalg.det(kernel[items_c, :][:, items_c])
            if max_v is None or max_v < cur_v:
                max_v = cur_v
                max_c = c
        items = items + [max_c]
    return items

In [ ]:
def dpp_rerank(user_index: int, item_indices: list[int], window: int):
    L = compute_kernel(user_index, item_indices)
    m = dict(zip(range(len(item_indices)), item_indices))

    R = list()
    W = np.arange(len(item_indices), dtype=np.int32)
    
    while len(W) != 0:
        M = greedy_approx_max(L[W,:][:,W], min(len(W), window))
        D = list()
        for ix in M:
            R.append(W[ix])
            D.append(W[ix])
        W = W[~np.isin(W, D)]
    
    return [int(m[r]) for r in R]

### Посмотрим, а оно вообще работает?

In [ ]:
TEST_USER = 1
raw_recommendations = TOP_USER_RECOMMENDATIONS[test_user]
dpp_recommendations = dpp_rerank(test_user, raw_recommendations, 5)

In [ ]:
TRACK_META.loc[raw_recommendations][:20]

In [ ]:
TRACK_META.loc[dpp_recommendations][:20]

### Переранжируем рекомендации с использованием DPP

При переранжировании возьмем 50 топовых (вряд ли пользователь потребит все)

In [ ]:
TOP_USER_RECOMMENDATIONS_DIVERSIFIED = list()
for u, recs in tqdm.tqdm(enumerate(TOP_USER_RECOMMENDATIONS)):
    TOP_USER_RECOMMENDATIONS_DIVERSIFIED.append(dpp_rerank(u, recs[:50], 10))

In [ ]:
TOP_USER_RECOMMENDATIONS_DIVERSIFIED = np.array(TOP_USER_RECOMMENDATIONS_DIVERSIFIED)

### Сохраним рекомендации

In [ ]:
def save_recs(recommendations: np.array, fhandle):
    result = list()
    for user_ix, tracks_ixs in enumerate(recommendations):
        user_id = USER_MAPPING[user_ix]
        tracks_ids = TRACK_META.loc[tracks_ixs]["track"].values.astype(int).tolist()

        result.append(json.dumps({
            "user": user_id,
            "tracks": tracks_ids
        }))
        
    fhandle.write("\n".join(result))

In [ ]:
with open("./result/recommendations_lfm.json", "w") as fh:
    save_recs(TOP_USER_RECOMMENDATIONS, fh)

with open("./result/recommendations_lfm_dpp.json", "w") as fh:
    save_recs(TOP_USER_RECOMMENDATIONS_DIVERSIFIED, fh)

# Подход #2. Разнообразие с использованием эвристики

В качестве простого альтернативного подхода - попробуем ограничить количество треков от одного артиста.
Идея здесь такая - чем больше уникальных артистов в нашей выдаче, тем разнообразнее наши рекомендации.

In [ ]:
K = 50
MAX_TRACKS_FROM_SAME_ARTIST = 2

ARTIST_DIVERSIFIED_RECOMMENDATIONS = list()
for user_index, recs_indices in tqdm.tqdm(enumerate(TOP_USER_RECOMMENDATIONS)):
    artists = dict()
    recs_ids = list()
    for rec_ix in recs_indices:
        if len(recs_ids) >= K:
            break
        
        track = TRACK_META.loc[rec_ix]
        artist = track['artist']

        if artist not in artists.keys():
            artists[artist] = 0
        artists[artist] += 1

        if artists[artist] <= MAX_TRACKS_FROM_SAME_ARTIST:
            recs_ids.append(int(track['track']))

    ARTIST_DIVERSIFIED_RECOMMENDATIONS.append({
        "user": USER_MAPPING[user_index],
        "tracks": recs_ids
    })


with open("./result/recommendations_lfm_auth.json", "w") as fh:
   for line in ARTIST_DIVERSIFIED_RECOMMENDATIONS:
       fh.write(json.dumps(line) + "\n")

In [ ]:
TRACK_META.set_index("track").loc[ARTIST_DIVERSIFIED_RECOMMENDATIONS[TEST_USER]["tracks"]][:20]